# Compare-Algorithm

In [80]:
import os

def writeToCSV(beeid, video, indirection, outdirection):
    # write to MaschineLearnedData.csv
    #print("Error: File does not exist!")
    attache = 'a'
    if not (os.path.isfile('./result.csv')):
        attache = 'w'
        with open('result.csv', attache) as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
            filewriter.writerow(["bee_id", "video", "in_direction", "out_direction"])
        attache = 'a'
                    
    with open('result.csv', attache) as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow([beeid, video, indirection, outdirection]) 
        
def formatFileName(filename):
    return filename.split('_')[-1].split('.')[0]

In [81]:
def compute(dicAutomated, dicManually): 
    """ compare data of both dictionaries """
    # take key of dicAutomated as long as it has a value -> if empty list: delete key from dic
    # kompare key value with key value of dicManually + delete compared key-value of both dic's

    _correctMatches = 0
    _allMatches = 0
    _inCorrectness = 0
    _outCorrectness = 0

    # iterate all videos
    for vidVal in list(dicManually):
        print('.', end='')
        # as long we have values in videos
        while dicManually[vidVal] != []:       
            currAutVal = []
            currManVal = dicManually[vidVal][0] # store bee_id

            # loop through dicManually Value to with same time_in name
            for bee in dicAutomated[vidVal]:
                # check if bee is inside
                if (bee[0] == currManVal[0]):
                    _allMatches += 1
                    inside = True
                    currAutVal = bee
                    # start simple compare
                    if (currAutVal[1] == currManVal[1] and currAutVal[2] == currManVal[2]):
                        _correctMatches += 1
                        writeToCSV(bee[0], vidVal, "OK", "OK")
                        _inCorrectness += 1
                        _outCorrectness += 1
                    elif (currAutVal[1] != currManVal[1] and currAutVal[2] != currManVal[2]):
                        writeToCSV(bee[0], vidVal, "mismatch", "mismatch")
                    elif (currAutVal[1] == currManVal[1] and currAutVal[2] != currManVal[2]):
                        writeToCSV(bee[0], vidVal, "OK", "mismatch")
                        _inCorrectness += 0.5
                    else:
                        writeToCSV(bee[0], vidVal, "mismatch", "OK")
                        _outCorrectness += 0.5
                else:
                    #print('Bee', beeid[0], 'hasn\'t been detected in both videos. Not compareable!')
                    writeToCSV(bee[0], vidVal, "n.a.", "n.a.")          

            del dicManually[vidVal][0]

    try:
        print(round(_correctMatches / _allMatches, 4) * 100, "% total correctness\n")
        print(round(_inCorrectness / _allMatches, 4) * 100, "% Ingoing correctness\n")
        print(round(_outCorrectness / _allMatches, 4) * 100, "% Outgoing correctness\n")
               
    except ZeroDivisionError:
        print("Nothing to compare!")

    print('Done. Please check result.csv file for more details.')
    

In [82]:
""" 
Precondition:
Fetch computed data and GroundTruthData from generated csv files and store in dictionaries.
"""

# read data from GroundTruthData.csv files
import csv
# use Dictionary to store data we want to compare
dicManuallyIn = {} # key(timestamp_in) - List
dicAutomatedIn = {} # key(times_in) - List

dicManuallyOut = {} # key(timestamp_out) - List
dicAutomatedOut = {} # key(times_out) - List

with open("MaschineLearnedData.csv", "r") as testFile:
    """ fetching data from MaschineLearnedData.csv file sorted by time_in as key """
    reader = csv.DictReader(testFile)
    for row in reader:
        # store video in dicAutomated
        if not (row['time_in'] in dicAutomatedIn):
            # add video to dic with his bees
            dicAutomatedIn[row['time_in']] = [[str(int(float(row['bee_id']))), row['in_direction'], row['out_direction'], row['time_out']]]
        else:
            #add other bees to the video key
            dicAutomatedIn[row['time_in']].append([str(int(float(row['bee_id']))), row['in_direction'], row['out_direction'], row['time_out']])
                        
with open("GroundTruthData.csv", "r") as testFile:
    """ fetching data from MaschineLearnedData.csv file sorted by timestamp_in as key """
    reader = csv.DictReader(testFile)
    for row in reader:
        # check row has an bee_id -> if nor go to next row
        if (row['bee_id'] != ""):
            # store video in dicManually 
            if not (formatFileName(row['timestamp_in']) in dicManuallyIn):
                # add video to dic with his bees
                dicManuallyIn[formatFileName(row['timestamp_in'])] = [[row['bee_id'], row['in_direction'], row['out_direction'], row['timestamp_out']]]
            else:
                #add other bees to the video key
                dicManuallyIn[formatFileName(row['timestamp_in'])].append([row['bee_id'], row['in_direction'], row['out_direction'], row['timestamp_out']])
        else:
            continue

            

with open("MaschineLearnedData.csv", "r") as testFile:
    """ fetching data from MaschineLearnedData.csv file sorted by time_out as key """
    reader = csv.DictReader(testFile)
    for row in reader:
        # store video in dicAutomated
        if not (row['time_out'] in dicAutomatedOut):
            # add video to dic with his bees
            dicAutomatedOut[row['time_out']] = [[str(int(float(row['bee_id']))), row['in_direction'], row['out_direction'], row['time_in']]]
        else:
            #add other bees to the video key
            dicAutomatedOut[row['time_out']].append([str(int(float(row['bee_id']))), row['in_direction'], row['out_direction'], row['time_in']])

with open("GroundTruthData.csv", "r") as testFile:
    """ fetching data from MaschineLearnedData.csv file sorted by timestamp_out as key """
    reader = csv.DictReader(testFile)
    for row in reader:
        # check row has an bee_id -> if nor go to next row
        if (row['bee_id'] != ""):
            # store video in dicManually 
            if not (formatFileName(row['timestamp_out']) in dicManuallyOut):
                # add video to dic with his bees
                dicManuallyOut[formatFileName(row['timestamp_out'])] = [[row['bee_id'], row['in_direction'], row['out_direction'], row['timestamp_in']]]
            else:
                #add other bees to the video key
                dicManuallyOut[formatFileName(row['timestamp_out'])].append([row['bee_id'], row['in_direction'], row['out_direction'], row['timestamp_in']])
        else:
            continue

In [83]:
""" Compare input of MaschineLearnedData.csv with GroundTruthData.csv to get just compareable tracks """

# delete values which are noch included in dicManually
for msData in list(dicAutomatedIn):
    if not (msData in dicManuallyIn):
        dicAutomatedIn.pop(msData)
        
# delete values which are not included in dicAutomated
for msData in list(dicManuallyIn):
    if not (msData in dicAutomatedIn):
        dicManuallyIn.pop(msData)

for msData in list(dicAutomatedOut):
    if not (msData in dicManuallyOut):
        dicAutomatedOut.pop(msData)
        
# delete values which are not included in dicAutomated
for msData in list(dicManuallyOut):
    if not (msData in dicAutomatedOut):
        dicManuallyOut.pop(msData)
        

In [84]:
""" print video-match """ 
#print(len(dicAutomatedIn))
#print(len(dicManuallyIn))
#print('###################')
#print(len(dicAutomatedOut))
#print(len(dicManuallyOut))

# create a new dic with the union of both dictionaries

dicManually = dicManuallyIn.copy()
for i in dicManuallyOut:
    if not (i in dicManuallyIn):
        dicManually[i] = dicManuallyOut[i]

dicAutomated = dicAutomatedIn.copy()
for i in dicAutomatedOut:
    if not (i in dicAutomatedIn):
        dicAutomated[i] = dicAutomatedOut[i]

#compute(dicAutomatedIn, dicManuallyIn)
#compute(dicAutomatedOut, dicManuallyOut)

compute(dicAutomated, dicManually)


..................................................................55.989999999999995 % correctness

63.449999999999996 % Ingoing correctness

67.94 % Outgoing correctness

Done. Please check result.csv file for more details.
